<a href="https://colab.research.google.com/github/Dan1lk/Analyzing_web_logs_from_csv/blob/main/analyzing_logs_from_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark py4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=1fba9790eefd3847222b0792b713fee90aedf57e1603f8ecf7d58b8dbb7be802
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, sum, to_date
spark = SparkSession.builder \
    .appName('web_logs_app') \
    .config('spark.master', 'local[*]') \
    .getOrCreate()

web_logs = spark.read.csv('web_server_logs.csv', header=True, inferSchema=True)
#добавляем столбец date в формате 'yyyy-MM-dd'
web_logs = web_logs.withColumn('date', to_date('timestamp', 'yyyy-MM-dd'))

# 10 самых активных ip

active_ip = web_logs.groupBy('ip').agg(count('*').alias('request_count')) \
     .orderBy('request_count', ascending=False).limit(10)
print("Top 10 active IP addresses: ")
active_ip.show()

# группировка по HTTP методу

group_by_http = web_logs.groupBy('method').agg(count('*').alias('method_count'))
print("Request count by HTTP method: ")
group_by_http.show()

# количество запросов с кодом ответа 404

count_404_query = web_logs.filter(web_logs.response_code == 404).groupBy('response_code') \
     .agg(count('*').alias('code_count'))
print(f"Number of 404 response codes: {count_404_query.first()['code_count']}")
print()

# группировка по дате и сумма размера ответов
group_by_date = web_logs.groupBy('date').agg(sum('response_size').alias('total_response_size')).orderBy('date').limit(15)
print('Total response size by day: ')
group_by_date.show()

spark.stop()

Top 10 active IP addresses: 
+---------------+-------------+
|             ip|request_count|
+---------------+-------------+
| 178.238.251.88|            2|
|  61.140.179.69|            2|
|  167.35.126.29|            2|
|160.202.253.117|            1|
| 122.72.233.188|            1|
| 148.43.131.173|            1|
|  19.62.199.241|            1|
| 51.222.202.133|            1|
|  38.23.165.204|            1|
| 187.128.164.26|            1|
+---------------+-------------+

Request count by HTTP method: 
+------+------------+
|method|method_count|
+------+------------+
|  POST|       25019|
|DELETE|       24943|
|   PUT|       24930|
|   GET|       25108|
+------+------------+

Number of 404 response codes: 24790

Total response size by day: 
+----------+-------------------+
|      date|total_response_size|
+----------+-------------------+
|2024-01-01|            2416994|
|2024-01-02|            2198263|
|2024-01-03|            2454239|
|2024-01-04|            2421939|
|2024-01-05|     